<a href="https://colab.research.google.com/github/adam-foley/Survivor/blob/main/survivor_data_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Surivor - Outwit, Outplay, Outlast

## Imports and functions

In [1]:
%pip install gender_guesser
%pip install pymysql

from bs4 import BeautifulSoup

import requests
import urllib.request
import time
import random
import requests
import random
import os
import pymysql
import time
import gender_guesser.detector as gender
import pandas as pd
import numpy as np

STATES = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'D.C.': 'DC', 'Florida': 'FL', 
          'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 
          'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 
          'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 
          'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 
          'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA.', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


SEASONS = ['Borneo', 'The_Australian_Outback', 'Africa', 'Marquesas', 'Thailand', 'The_Amazon', 'Pearl_Islands', 'All-Stars', 'Vanuatu', 'Palau', 'Guatemala','Panama', 'Cook_Islands', 'Fiji'] 

def getSeasonURL(season_num):
  wiki_url = 'https://en.wikipedia.org/wiki/Survivor:_'
  return wiki_url + SEASONS[ season_num - 1 ]

def getSurvivorId(first_name, last_name):
  first_name = "'"+first_name+"'"
  last_name = "'"+last_name.strip(" ")+"'"
  query = "Select survivor_id from survivor_data.Players where first_name ="+first_name+" and last_name="+last_name+";"
  cur.execute(query)
  id = cur.fetchone()[0]
  return str(id)

def getSurvivorLastName(first_name):
  first_name = "'"+first_name+"'"
  season = "'"+str(len(SEASONS))+"'"
  query = '''Select p.last_name from survivor_data.Players p  
                                left join (Select s.survivor_id 
                                          from Seasons s 
                                          left join Players p on s.survivor_id = p.survivor_id and s.season_id ='''+season+'''
                                          where p.first_name ='''+first_name+'''
                                          ) s on p.survivor_id = s.survivor_id
                                          
                                where p.survivor_id = s.survivor_id;'''
                                
  cur.execute(query)
  id = cur.fetchone()[0]
  return str(id)

def executeQuery(query):
  cur.execute(query)

def commitQuery():
  conn.commit()

def findStateInString(s):
  word = ''
  for char in s:
    if word in states:
      return word
    else:
      word += char

def addQuotes(s):
  return " '"+s+"' "

def getGender(name):
  d = gender.Detector()
  g = d.get_gender(name).split("_")

  if len(g) == 2:
    gn = g[-1]
  else:
    gn = g[0]
    
  if gn == "male":
    return 'M'
  else:
    return 'F'

def getPlayerTableInfoRandom(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  table = soup.find_all('table', {'class': 'wikitable'})[0]
  table_rows = table.find_all('tr')

  rows = []
  for tr in table_rows:
      td = tr.find_all('td')
      td_row = [i.text for i in td]
      rows.append(td_row[:-1])
      random.shuffle(rows)

  return rows

def getPlayerTableInfo(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  table = soup.find_all('table', {'class': 'wikitable'})[0]
  table_rows = table.find_all('tr')

  rows = []
  for tr in table_rows:
      td = tr.find_all('td')
      td_row = [i.text for i in td]
      rows.append(td_row[:-1])

  return rows

     |████████████████████████████████| 379 kB 7.4 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 


## AWS Login

In [2]:
  succeess = False

  while not succeess:
    password = input("Password: ");
    try:
      conn = pymysql.connect(
          host='aws-survivor.cpapv35x7rsh.us-east-2.rds.amazonaws.com',
          port=int(3306),
          user="admin",
          passwd=password,
          db="survivor_data",
          charset='utf8mb4')
      cur = conn.cursor()
      succeess = True
    except:
      print("Wrong Password")
  print("Connection Made")

Password: Frederickwilliams121234
Connection Made


## Adding Players to Players Table

In [4]:
url = getSeasonURL(len(SEASONS))
rows =getPlayerTableInfoRandom(url)
print(rows)

for row in rows:
  if len(row) !=0:
    r_split = row[0].split(",")

    if len(r_split) == 3:
      name_age = r_split[0]
      first_name = name_age[:-2].split(" ", 1)[0]
      last_name  = name_age[:-2].split(" ", 1)[1]
      age = name_age[-2:]
      gen = getGender(first_name)
      state = r_split[-1].strip("\n")[1:]

      print(first_name, last_name, age, gen, state)

      try:
        query = "INSERT INTO `survivor_data`.`Players` (`first_name`, `last_name`, `sex`, `state`) VALUES ("+addQuotes(first_name)+","+addQuotes(last_name)+","+addQuotes(gen)+","+addQuotes(STATES[state])+");"
        print(query)
        #executeQuery(query)
      except:
        print("Error")

#commitQuery()

[['Edgardo Rivera28, Miami Beach, Florida\n', 'Moto\n', 'Ravu\n'], ['Michelle Yi23, Cincinnati, Ohio\n', 'Ravu\n', 'Moto\n', 'Bula Bula\n'], ['Rita Verreos38, San Antonio, Texas\n', 'Ravu\n', '\n', '\n'], ['Cassandra Franklin42, Los Angeles, California\n', 'Moto\n', 'Moto\n'], ['Anthony Robinson32, Compton, California\n', 'Ravu\n', 'Ravu\n', '\n'], ['James "Rocky" Reid28, Los Angeles, California\n', 'Ravu\n', 'Ravu\n', '\n'], ['Mookie Lee25, Chicago, Illinois\n', 'Ravu\n', 'Ravu\n'], ['Kenward "Boo" Bernis34, Lafayette, Louisiana\n', 'Moto\n', 'Moto\n'], ['Jessica deBen27, Los Angeles, California\n', 'Ravu\n', '\n', '\n'], ['Stacy Kimball27, Boulder, Colorado\n', 'Moto\n', 'Moto\n'], ['Earl Cole35, Santa Monica, California\n', 'Ravu\n', 'Moto\n'], ['Erica Durousseau27, Baton Rouge, Louisiana\n', 'Ravu\n', '\n', '\n'], ['Lisette "Lisi" Linares36, Los Angeles, California\n', 'Moto\n', 'Ravu[b]\n', '\n'], ['Yau-Man Chan54, Martinez, California\n', 'Ravu\n', 'Moto\n'], ['Andria "Dreamz" He

## Add Players to Seasons Table

In [6]:
season = len(SEASONS)
url = getSeasonURL(season)
rows =getPlayerTableInfoRandom(url)
print(rows)

for row in rows:
  if len(row) !=0:
    r_split = row[0].split(",")

    if len(r_split) == 3:
      name_age = r_split[0]
      first_name = name_age[:-2].split(" ", 1)[0]
      last_name  = name_age[:-2].split(" ", 1)[1]
      age = name_age[-2:]


      try:
        survivor_id = getSurvivorId(first_name, last_name)
        query = "INSERT INTO `survivor_data`.`Seasons` (season_id, survivor_id, age)"+" VALUES('"+str(season)+"','"+str(survivor_id)+"','"+str(age)+"');"
        
        print(query)
        #executeQuery(query)
      except:
        print(first_name, last_name, age)

#commitQuery()

[['Earl Cole35, Santa Monica, California\n', 'Ravu\n', 'Moto\n'], ['Gary Stritesky55, Ramsey, Minnesota\n', 'Moto\n', '\n', '\n'], [], ['James "Rocky" Reid28, Los Angeles, California\n', 'Ravu\n', 'Ravu\n', '\n'], ['Edgardo Rivera28, Miami Beach, Florida\n', 'Moto\n', 'Ravu\n'], ['Andria "Dreamz" Herd25, Wilmington, North Carolina\n', 'Moto\n', 'Ravu\n'], ['Erica Durousseau27, Baton Rouge, Louisiana\n', 'Ravu\n', '\n', '\n'], ['Lisette "Lisi" Linares36, Los Angeles, California\n', 'Moto\n', 'Ravu[b]\n', '\n'], ['Stacy Kimball27, Boulder, Colorado\n', 'Moto\n', 'Moto\n'], ['Anthony Robinson32, Compton, California\n', 'Ravu\n', 'Ravu\n', '\n'], ['Cassandra Franklin42, Los Angeles, California\n', 'Moto\n', 'Moto\n'], ['Yau-Man Chan54, Martinez, California\n', 'Ravu\n', 'Moto\n'], ['Rita Verreos38, San Antonio, Texas\n', 'Ravu\n', '\n', '\n'], ['Sylvia Kwan52, Ross, California\n', 'Ravu[a]\n', '\n', '\n'], ['Jessica deBen27, Los Angeles, California\n', 'Ravu\n', '\n', '\n'], ['Michelle Yi2

## Add Votes for in Season Table

In [ ]:
a = [ 'Nathan', 'Oscar', 'Rebekah']
v = [ 'Candice', 'Cecilia', 'Adam']
season = len(SEASONS)


for player in a:
  survivor_id = getSurvivorId(player,getSurvivorLastName(player))
  query = "UPDATE `survivor_data`.`Seasons` SET `vote_adam` = 1 WHERE `season_id` ='"+str(season)+"' and survivor_id ='"+survivor_id+"';"
  #executeQuery(query)
  print(query)

for player in v:
  survivor_id = getSurvivorId(player,getSurvivorLastName(player))
  query = "UPDATE `survivor_data`.`Seasons` SET `vote_victoria` = 1 WHERE `season_id` ='"+str(season)+"' and survivor_id ='"+survivor_id+"';"
  #executeQuery(query)
  print(query)

#commitQuery()

UPDATE `survivor_data`.`Seasons` SET `vote_adam` = 1 WHERE `season_id` ='13' and survivor_id ='252';
UPDATE `survivor_data`.`Seasons` SET `vote_adam` = 1 WHERE `season_id` ='13' and survivor_id ='247';
UPDATE `survivor_data`.`Seasons` SET `vote_adam` = 1 WHERE `season_id` ='13' and survivor_id ='250';
UPDATE `survivor_data`.`Seasons` SET `vote_victoria` = 1 WHERE `season_id` ='13' and survivor_id ='257';
UPDATE `survivor_data`.`Seasons` SET `vote_victoria` = 1 WHERE `season_id` ='13' and survivor_id ='251';
UPDATE `survivor_data`.`Seasons` SET `vote_victoria` = 1 WHERE `season_id` ='13' and survivor_id ='241';


## Set Players Place in the Season

In [ ]:
season = len(SEASONS)
url = getSeasonURL(season)
rows =getPlayerTableInfo(url)
rows = rows[1:]
place = len(rows)

print(rows)

for row in rows:
  if len(row) !=0:
    r_split = row[0].split(",")

    if len(r_split) == 3:
      name_age = r_split[0]
      first_name = name_age[:-2].split(" ", 1)[0].strip(" ")
      last_name  = name_age[:-2].split(" ", 1)[1].strip(" ").strip("'")

      try:
        print(first_name)
        survivor_id = getSurvivorId(first_name, last_name)
        query = "UPDATE `survivor_data`.`Seasons` SET place ="+ str(place) +" WHERE `season_id` ='"+str(season)+"'and `survivor_id` ='"+survivor_id+"';"
        print(query)
        executeQuery(query)

      except:
        print("Error: ", first_name, last_name) 
        
  place -= 1   

commitQuery()

[['Sekou Bunch45, Los Angeles, California\n', 'Manihiki\n', '\n', '\n', '\n'], ['Virgilio "Billy" Garcia36, New York City, New York\n', 'Aitutaki\n', '\n', '\n', '\n'], ['Cecilia Mansilla29, Oakland, California\n', 'Aitutaki\n', 'Aitutaki\n', '\n', '\n'], ['John "J.P." Calderon30, Long Beach, California\n', 'Aitutaki\n', 'Rarotonga\n', '\n', '\n'], ['Stephannie Favor35, Columbia, South Carolina\n', 'Manihiki\n', 'Rarotonga\n', '\n', '\n'], ['Anh-Tuan "Cao Boi" Bui42, Christiansburg, Virginia\n', 'Puka Puka\n', 'Aitutaki\n', '\n', '\n'], ['Cristina Coria35, Santa Monica, California\n', 'Aitutaki\n', 'Rarotonga\n', '\n', '\n'], ['Jessica Smith27, Chico, California\n', 'Rarotonga\n', 'Aitutaki\n', '\n', '\n'], ['Brad Virata29, Santa Monica, California\n', 'Puka Puka\n', 'Rarotonga\n', 'Rarotonga\n', '\n'], ['Rebecca Borman34, Laurelton, New York\n', 'Manihiki\n', 'Rarotonga\n', 'Rarotonga\n', '\n'], ['Jenny Guzon-Bae36, Lake Forest, Illinois\n', 'Puka Puka\n', 'Rarotonga\n', 'Rarotonga\n'

## Getting Challenge Winners

In [ ]:
response = requests.get(getSeasonURL(7))
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table', {'class': 'wikitable'})[1]
table_rows = table.find_all('tr')

rows = []
for tr in table_rows:
    td = tr.find_all('td')
    td_row = [i.text for i in td]
    if len(td_row) != 0:
      rows.append(td_row)

print(rows)

count = 0

for epi in rows:
  if count >= 8:
    reward = 'N/A'
    immunity = 'N/A'

    print(epi)

    if epi[-1].strip("\n") != 'Runner-up' or epi[-1].strip("\n") != 'Sole Survivor':
      if epi[-1][:2] == '13' or epi[-1][:2] == '14':
        immunity = epi[-3].strip("\n")
      elif len(epi) >= 4:
        reward = epi[-4].strip("\n")
        immunity = epi[-3].strip("\n")
    
    print(reward, immunity)
  count +=1